<a href="https://colab.research.google.com/github/bhagyasharma/nlp_commonsenseqa/blob/master/evaluate_results_bert_commonsenseqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%%shell
pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 122kB 3.9MB/s 


In [0]:
# train model
# this part can be skipped if you already have your model
!python gdrive/My\ Drive/Colab\ Notebooks/train_csqa.py   --bert_model bert-base-uncased  --do_lower_case   --data_dir gdrive/My\ Drive/Colab\ Notebooks/qtoken_3_data   --num_choices 3 --train_batch_size 16   --learning_rate 2e-5   --num_train_epochs 1.0   --max_seq_length 128   --output_dir /tmp/csqa_output/   --gradient_accumulation_steps 4

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
03/14/2019 17:41:58 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False
03/14/2019 17:41:59 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/14/2019 17:41:59 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/14/2019 17:41:59 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretraine

In [1]:
import torch
import csv
import os
import sys
import logging
import numpy as np
import pandas as pd
from pytorch_pretrained_bert.modeling import BertForMultipleChoice, BertConfig
from pytorch_pretrained_bert.tokenization import BertTokenizer
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
config_file = '/tmp/csqa_output/bert_config.json'
num_answer_choices = 3
pytorch_model = '/tmp/csqa_output/pytorch_model.bin'
file_csv = '/content/gdrive/My Drive/Colab Notebooks/qtoken_3_data/val.csv'
is_training_set = True
# results saved as 'predictions.csv' & 'questions.jsonl' at end


# Load a trained model and config that you have fine-tuned
config = BertConfig(config_file)
model = BertForMultipleChoice(config, num_choices=num_answer_choices)
model.load_state_dict(torch.load(pytorch_model))

In [0]:
## same code from train_csqa.py to set up classes
class CSQAExample(object):
    """A single training/test example for the CSQA dataset."""
    def __init__(self,
                 csqa_id,
                 question,
                 choice_0,
                 choice_1,
                 choice_2,
                 choice_3 = None,
                 choice_4 = None,
                 label = None):
        self.csqa_id = csqa_id
        self.question = question
        self.choices = [
            choice_0,
            choice_1,
            choice_2,
            choice_3,
            choice_4
        ]
        self.choices = list(filter(None.__ne__, self.choices))
        self.label = label

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        l = [
            "csqa_id: {}".format(self.csqa_id),
            "question: {}".format(self.question),
            "choice_0: {}".format(self.choices[0]),
            "choice_1: {}".format(self.choices[1]),
            "choice_2: {}".format(self.choices[2]),
            "choice_3: {}".format(self.choices[3]),
            "choice_4: {}".format(self.choices[4])
        ]

        if self.label is not None:
            l.append("label: {}".format(self.label))

        return ", ".join(l)


class InputFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label


def read_csqa_examples(input_file, num_choices, is_training):
    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        lines = []
        for line in reader:
            if sys.version_info[0] == 2:
                line = list(unicode(cell, 'utf-8') for cell in line)
            lines.append(line)

    if is_training and lines[0][-1] != 'label':
        raise ValueError(
            "For training, the input file must contain a label column."
        )

    examples = []
    for line in lines[1:]:
        #logger.info('Question Id: ' + str(line[0]))
        label_ = None
        c_3 = None
        c_4 = None
        
        if is_training:
            label_ = line[7] #default is 3 answer choices
        if num_choices == 4:
            c_3 = line[7]
            if is_training:
                label_ = line[8]
        elif num_choices == 5:
            c_3 = line[7]
            c_4 = line[8]
            if is_training:
                label_ = line[9]

        examples.append(
          CSQAExample(
              csqa_id = line[1],
              question = line[3],

              choice_0 = line[4],
              choice_1 = line[5],
              choice_2 = line[6],
              choice_3 = c_3,
              choice_4 = c_4,
              label = int(label_) if is_training else None))

    return examples

def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 is_training):
    """Loads a data file into a list of `InputBatch`s."""

    # Use this formatting for tokenization
    # Each choice will be tokenized accordingly:
    # - [CLS] question [SEP] choice_1 [SEP]
    # - [CLS] question [SEP] choice_2 [SEP]
    # - [CLS] question [SEP] choice_3 [SEP]
    # The model will output a single value for each input. To get the
    # final decision of the model, we will run a softmax over these 3
    # outputs.
    features = []
    for example_index, example in enumerate(examples):
        context_tokens = tokenizer.tokenize(example.question)

        choices_features = []
        for choice_index, choice in enumerate(example.choices):
            # We create a copy of the context tokens in order to be
            # able to shrink it according to choice_tokens
            context_tokens_choice = context_tokens[:]
            choice_tokens = tokenizer.tokenize(choice)
            # Modifies `context_tokens_choice` and `choice_tokens` in
            # place so that the total length is less than the
            # specified length.  Account for [CLS], [SEP], [SEP] with
            # "- 3"
            
            _truncate_seq_pair(context_tokens_choice, choice_tokens, max_seq_length - 3)

            tokens = ["[CLS]"] + context_tokens_choice + ["[SEP]"] + choice_tokens + ["[SEP]"]
            segment_ids = [0] * (len(context_tokens_choice) + 2) + [1] * (len(choice_tokens) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = example.label
        if example_index < 5:
            logger.info("*** Example ***")
            logger.info("csqa_id: {}".format(example.csqa_id))
            for choice_idx, (tokens, input_ids, input_mask, segment_ids) in enumerate(choices_features):
                logger.info("choice: {}".format(choice_idx))
                logger.info("tokens: {}".format(' '.join(tokens)))
                logger.info("input_ids: {}".format(' '.join(map(str, input_ids))))
                logger.info("input_mask: {}".format(' '.join(map(str, input_mask))))
                logger.info("segment_ids: {}".format(' '.join(map(str, segment_ids))))
            if is_training:
                logger.info("label: {}".format(label))

        features.append(
            InputFeatures(
                example_id = example.csqa_id,
                choices_features = choices_features,
                label = label
            )
        )

    return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
        return
    if len(tokens_a) > len(tokens_b):
        tokens_a.pop()
    else:
        tokens_b.pop()

def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
device = torch.device("cuda")
n_gpu = torch.cuda.device_count()
torch.manual_seed(42)
if n_gpu > 0:
  torch.cuda.manual_seed_all(42)
  
model.to(device)
if n_gpu > 1:
  model = torch.nn.DataParallel(model)
model.to(device)

# specify val.csv or test.csv here (will need to set is_training to False for test set)
eval_examples = read_csqa_examples(file_csv, 3, is_training_set)
eval_features = convert_examples_to_features(eval_examples, tokenizer, 128, True)
logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
all_input_ids = torch.tensor(select_field(eval_features, 'input_ids'), dtype=torch.long)
all_input_mask = torch.tensor(select_field(eval_features, 'input_mask'), dtype=torch.long)
all_segment_ids = torch.tensor(select_field(eval_features, 'segment_ids'), dtype=torch.long)
questions_ids = [f.example_id for f in eval_features]
all_label = torch.tensor([f.label for f in eval_features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=16)

model.eval()
eval_accuracy = 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions = []
for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids, segment_ids, input_mask)
    
    logits = logits.detach().cpu().numpy()
    outputs = np.argmax(logits, axis=1).tolist()
    predictions.extend(outputs)
    label_ids = label_ids.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += input_ids.size(0)

eval_accuracy = eval_accuracy / nb_eval_examples

result = {'eval_accuracy': eval_accuracy}
logger.info("***** Eval results *****")
for key in sorted(result.keys()):
    logger.info("  %s = %s", key, str(result[key]))

03/14/2019 18:57:15 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/14/2019 18:57:15 - INFO - __main__ -   *** Example ***
03/14/2019 18:57:15 - INFO - __main__ -   csqa_id: 2e601d898d9b2ce9dc0bacc404e753e7
03/14/2019 18:57:15 - INFO - __main__ -   choice: 0
03/14/2019 18:57:15 - INFO - __main__ -   tokens: [CLS] if your opponents in a marathon is very fast it is easy to fall what ? [SEP] slowly [SEP]
03/14/2019 18:57:15 - INFO - __main__ -   input_ids: 101 2065 2115 7892 1999 1037 8589 2003 2200 3435 2009 2003 3733 2000 2991 2054 1029 102 3254 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [14]:
len(predictions) #verify this is same as number of examples

938

In [15]:
##Create dataframe and save to csv & jsonl for test set
predictions = pd.DataFrame(
    {'id': questions_ids,
     'prediction': predictions
    })

answer_match = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E'}
predictions['prediction'] = predictions['prediction'].apply(lambda x: answer_match.get(x))
predictions.rename(columns={'prediction':'A;B;C;D;E'}, inplace=True)
predictions

,id,A;B;C;D;E
0,2e601d898d9b2ce9dc0bacc404e753e7,B
1,a265e3719e2eaf39b8fe6c901717303e,B
2,65e7a36ab495298a8a2431a3887a8770,A
3,4aa0a4d3b7590e9a91aed7e7c54aa364,A
4,65432eb6e617514d863a465f38865fde,B
5,4f79dd4b1c99219c526e23e55f88ea19,A
6,00d2de955e68c347a252d267826bee0b,C
7,7c67ad88467380f15575908e70cbbaf3,C
8,15484961c7e71fc6701fa2583050ba2c,B
9,8959f7fd535adcbfa7028dc13af3dcb2,C


In [0]:
predictions.to_csv('predictions.csv', mode='w', encoding='utf-8', index=False)

In [0]:
predictions.rename(columns={'A;B;C;D;E': 'answerKey'}, inplace=True)
data_dict = predictions.to_dict('records')

with open('questions.jsonl', 'w') as outfile:
    for k in data_dict:
        json.dump(k, outfile)
        outfile.write('\n')